In [9]:
dimension = 6
perimeter = 50
sensitivity = 1.7
jsonfilename = "noname.json"

In [19]:
# UNINTERESTING TOOLS
from random import randint, seed


def shortestVector(matrix): 
    """
    Returns the 3-tuple: norm of the shortest vector, the shortest vector and its index.
    """
    return sorted([(matrix[idx].norm().n(), matrix[idx], idx) for idx in range(matrix.nrows())])[0]

def randomMatrix(dimension, per) -> matrix:
    list = [randint(-per, per) for _ in range(dimension**2)]
    M = matrix(ZZ, dimension, dimension, list)
    while M.rank() != dimension:
        list = [randint(-per, per) for _ in range(dimension**2)]
        M = matrix(ZZ, dimension, dimension, list)
    return M

def gram_matrix(matrix) -> matrix:
    return matrix * matrix.transpose()

def matrix_to_list(A) -> list:
    return [[int(A.list()[row * A.ncols() + col]) for col in range(A.ncols())] for row in range(A.nrows())]

def vector_to_list(vector) -> list:
    return [float(num) for num in vector.list()]


def into_dict(B, G, x, solutions) -> dict:
    result = {}
    result["B"] = B
    result["G"] = G
    result["lincomb_LLL"] = x
    result["lincomb_cube"] = solutions
    result["sv_LLL"] = x*B
    result["sv_cube"] = vector(solutions)*B
    result["lincomb_diff"] = x - vector(solutions)
    result["Diff impact"] = multiply_B_by_diff(G, x - vector(solutions))[0]
#     result["Diff impact sum"] = multiply_B_by_diff(B, x - vector(solutions))[1]
#     result["LLL.norm"] = vector(x*B).norm().n(digits=5)
#     result["cube.norm"] = (vector(solutions)*B).norm().n(digits=5)
    return result

In [18]:
### batshit cool math
def innerComparsion(dimension, perimeter, B, B_red, v_min, x, G,i) -> dict:
    solutions = find_real_minimum(G, i, x[i])
    for j in range(dimension - 1):  # zbytecny forcyklus???
        difference = x[j] - solutions[j]
        if abs(difference) > sensitivity: 
            if all([sol == 0 for sol in solutions]):# FIME fuj
                break  
            return into_dict(B, G, x, solutions)
        
def find_real_minimum(G, i, x) -> vector:
    matrixA = matrix(G.nrows() - 1, G.nrows() - 1, 0)
    matrixB = matrix(G.nrows() - 1, 1, 0)
    matrixA[0,0] = 1
    a, b = 0, 0
    for row in range(G.nrows()):
        if row != i:
            matrixA[a] = [G[row, j] for j in range(len(G[row])) if j != i]
            matrixB[b] = sum([x * G[row,j] for j in range(len(G[row])) if j == i])
            a += 1
            b += 1
    # insert indices
    result = (matrixA.solve_right((-1)*matrixB)).list()
    result.insert(i,x)
    return vector(result).n(digits=5) 


def multiply_B_by_diff(B, diff) -> (matrix, float):
    B = matrix_to_list(B)
    n = len(B)
    diff = vector_to_list(diff)
    for i in range(n):
        for j in range(n):
            B[i][j] = round(B[i][j]*  diff[i] * diff[j], 2)
    return matrix(B), round(sum(sum(B, [])), 2)

In [17]:
### output formatting
import json

def format_data(output_data):
    for dic in output_data:
        for key, value in dic.items():
            if isinstance(value, sage.matrix.matrix_integer_dense.Matrix_integer_dense):
                dic[key] = matrix_to_list(value)
            else:
            # FIXME here I assume that output_data consits only of matrices and one-line entries (vectors, free modules)
            # it IS going to break if I try to save integers/floats
                dic[key] = vector_to_list(value)
    return output_data

def into_json(output_data):
    out_file = open(jsonfilename, "w+") 
    json.dump(output_data, out_file) 
    out_file.close() 

def from_json(input_data):
    out_file = open(jsonfilename)
    return json.load(out_file) # close it maybe?

def print_dict(dictionary) -> None:
    for pair in dictionary.items():
        print(pair[1], ": ", pair[0])
    print()
    
    

In [22]:
# MAIN 
def generate_new_examples(iterations) -> None:
    output_data = []
    for _ in range(iterations):
        B = randomMatrix(dimension, perimeter)
        data = abnormality_check(B)
        if data != None: #FIXME terrible, can I get rid of these ifs somehow?
            output_data.append(abnormality_check(B))
    for dictionary in output_data:
        print_dict(dictionary)
    into_json(format_data(output_data))


def abnormality_check(B) -> dict:
    B_red = B.LLL()
    v_min = shortestVector(B_red)[1]
    x = B.solve_left(v_min) # find linear combination corresponding to shortest vector found by LLL
    G = gram_matrix(B)
    for i in range(G.nrows()):
        data = innerComparsion(dimension, perimeter, B, B_red, v_min, x, G,i) # terrible
        if data != None:
            return data

In [23]:
generate_new_examples(1000)

[-11 -20  47 -44 -15  16]
[-30 -42 -39  44 -44 -17]
[ 12  -2   0  26  43  47]
[  1 -20 -40 -33  34   2]
[  1  18   5 -20 -28 -39]
[ 15 -35   5  42 -27  32] :  B
[ 5147 -2211 -1129  -517   540  -161]
[-2211  8346 -1823  -612    34  3317]
[-1129 -1823  4882   750 -3581  1685]
[ -517  -612   750  4250  -929 -1725]
[  540    34 -3581  -929  3055 -1922]
[ -161  3317  1685 -1725 -1922  4992] :  G
(-4, -5, -16, -2, -18, 1) :  lincomb_LLL
(-3.5051, -4.4767, -14.175, -1.6195, -15.810, 1.0000) :  lincomb_cube
(-3, 3, 2, 8, 1, -1) :  sv_LLL
(0.32111, -0.71875, 0.57874, 0.33545, 0.62500, -0.86914) :  sv_cube
(-0.49488, -0.52333, -1.8245, -0.38050, -2.1899, 0.00000) :  lincomb_diff
[  1260.54   -572.62   -1019.4    -97.35    585.21       0.0]
[  -572.62   2285.76  -1740.65   -121.87     38.96      -0.0]
[  -1019.4  -1740.65  16251.63    520.67 -14307.75      -0.0]
[   -97.35   -121.87    520.67    615.32   -774.08       0.0]
[   585.21     38.96 -14307.75   -774.08  14650.28       0.0]
[      0.0  